# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** <br><br>
**Description of the built CNN-RNN architecture in detail:**

1. **CNN Encoder:**
    - `Transfer Learning used` in this project where we have considered the `ResNet50 architecture`.
    - `ResNet50` is a `pre-trained` version of the network trained on more than a million images from the ImageNet database.
    - This `ResNet50 architecture (with the final fully-connected layer removed)` is `used in the encoder` to `extract the features` from the batch of pre-processed images.
    - `The output is then flattened to a vector, before being passed through a Linear layer (fully-connected layer at the end)` to `transform the feature vector to have the same size as the word embeddings.`<br><br>

2. **RNN Decoder:**
    - `Consists of 1 embedding layer` to `transform the input image features and captions into embeddings`. 
    - `LSTM takes LSTM inputs as a torch tensor obtained after concatenating features and embedded captions`, so that the model can find relationships between word tokens better.
    - `Consists of 1 LSTM layer (num_layers= 1)` with a possibility to extend the number of LSTM layers and add the dropout layers between them. For the mentioned architecture, the `default dropout is set to 0 (i.e. no dropout)`, as it seems to perform well with the mentioned configuration.
    - `Finally, 1 fully-connected linear layer to generate outputs.`.<br><br>
    - A `sample function is written` to `transform the outputs generated into the vocabulary keys`.<br><br>
     **Sample Function (within Decoder):**<br>
     - `Embedded features of an image (output of encoder)` is `passed through the decoder sample function`. This sample function uses LSTM to generate outputs in the form of a list of predicted word token indices of subsequent words until the end of the caption or max caption length is reached. 
     - `Reverse mapping of the predicted indices of these word tokens is done to retrieve the words` (using clean_sentence() function - 3_Inference.ipynb) and `then the corresponding clean sentence caption is generated`.<br><br>

**Selecting the values of variables (including hyperparameters) in Task 1, and the reasons behind selections:**

- **batch_size** - `Batch size is the number of image-caption pairs used to amend the model weights in each training step.` The batch size of each training batch was changed and `iterated for a small number of steps within epoch with varying sizes as 32, 64, 128 and 256.` For the current model architecture, `the batch size of 128 was selected based on the following reasons`:<br><br>
    - `A batch size not too large but at the same time not too small.` It should be `able to generalize the model better`. 
    - Slightly larger batch_size: `Helps it train the model faster on GPU`. 
    - The model has thousands of images in the training data, so `we can afford to use a batch_size of 128 which gives us a decent number of backpropagation steps to update model weights and thereby deliver better performance over epochs`.<br><br>
- **vocab_threshold** - `The minimum word count threshold was chosen to be 4, because this seemed to be a reasonable size for the vocabulary`. Moreover, the rare words were even quite rarer in their unique appearances; resulting in not a vast increase in vocabulary. Hence, the choice.<br><br>    
- **vocab_from_file** - `A Boolean that decides whether to load the vocabulary from a file.` `Based on the new vocab_threshold, a new vocabulary will be generated. Hence, vocab_from_file was set to False in order to generate new vocabulary`.<br><br>
- **embed_size** - `The dimensionality of the image and word embeddings was chosen to be 500` as for some studied architectures, 300-500 seemed to be an ideal size.<br><br>
- **hidden_size** - `The number of features in the hidden state of the RNN decoder was chosen to be 512`. This is because we want the LSTM to include the nuances in the flow of subsequent word token predictions and at the same time, it shouldn't be too large to overfit the model. <br><br>
- **num_epochs** - `Have taken the recommended number of epochs into account while training the model (num_epochs= 3)`. On evaluating the performance of the model, and on the basis of availability of GPU resources, it was deduced that these many epochs were sufficient for a decent performance. <br><br>
- **save_every** - Determines how often to save the model weights. Set `save_every=1, to save the model weights after each epoch`. This way, after the ith epoch, the encoder and decoder weights will be saved in the models/ folder as encoder-i.pkl and decoder-i.pkl, respectively.<br><br>
- **print_every** - Determines how often to print the batch loss to the Jupyter notebook while training. Chosen value of `print_every =100 (default- recommended)`.<br><br>
- **log_file** - The name of the text file containing - for every step - how the loss and perplexity evolved during training. File log can be found in the `"training_log.txt"` file within the directory.<br><br>



### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** <br><br>
**Selection of transforms in 'transfrom_train':**<br><br>
`The default data augmentation transforms were included in the 'transform_train'` as they seemed to be quite apt and relevant to the given model architecture and the purpose. 

- `The images in the dataset were resized to an image with smaller edge to 256, and were randomly cropped to get images of size 224 x 224.` 
- `Also, apart from the other data transforms performed, these images were converted into tensors and appropriate normalization was performed on all images (to feed them to the respective custom pre-trained model.)`

*The train transforms were as follows:*
- (Defining 'transform_train' to pre-process the training images).
        transform_train = transforms.Compose([ 
            transforms.Resize(256),                          # smaller edge of image resized to 256
            transforms.RandomCrop(224),                      # get 224x224 crop from random location
            transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
            transforms.ToTensor(),                           # convert the PIL Image to a tensor
            transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                                 (0.229, 0.224, 0.225))])


**Reasons why these train transforms are a good choice for the mentioned architecture:**
- `Usually, data augmentation helps in introducing the model to different transformed variants of an image by increasing the size of the dataset. But it should be noted that in PyTorch, 'on the fly' transforms are applied on each item in the dataset one by one and not adding to the size of the dataset.` <br><br>

- For example, data augmentation transforms (e.g. RandomHorizontalFlip) are actually applied on each item in the dataset one by one (randomly based on the probability) and not adding to the size of the dataset, thereby not augmenting the size of the dataset. `Although there is work-around to augment the data, by default, in PyTorch 'on the fly' transforms operations are applied to the original images at every batch generation. Meaning, the dataset is left unchanged, only a batch of images are copied and transformed every iteration when we perform these 'on the fly' transforms.` <br><br>

- `Also, there is no need to augment the data (additional transformed images concatenated to the existing dataset) by increasing the size of the original dataset`. This is `because` <br>
`(i) we have enough images in our training dataset to train the model` and <br>
`(ii) such augmentation might lead to increased size of dataset, thereby leading to increased training time`.<br><br>

- Hence, the default train transforms were a good choice for the mentioned architecture.<br>


### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** <br><br>
**Selecting the trainable parameters in CNN-RNN architecture, and reasons for their selection:**

1. **In Encoder:**<br>
    - Since we are using Transfer Learning in the encoder, `the parameters of all the layers except the last embedding layer are frozen to avoid training of these pre-trained parameters of the ResNet50 architecture`. 
    - **Only updated the parameters of the embedding layer (embed) in the encoder**. Thus, setting the training and update of parameters of all the other layers to False, making only the last embed layer trainable. <br><br>

2. **In Decoder:**<br>
    - **All the decoder parameters are set to trainable** because `the decoder has a relatively low number of parameters`, which are necessary to be trained. 
    - `Since they are small in number`, it might necessarily `not result in overfitting`.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

**For optimization: Adam optimizer was chosen.**<br>
`Reasons:`<br>
- `For optimization, I chose Adam over SGD with Momentum.` This is because Adam optimizer combines the advantages of two SGD extensions — Root Mean Square Propagation (RMSProp) and Adaptive Gradient Algorithm (AdaGrad), and computes individual adaptive learning rates for different parameters. 
- `Learning rate` was an important hyperparameter that required some tuning within the Adam optimizer. `Adam optimizer seemed to have performed better` for the best network architecture `when implemented with an optimal learning rate of 0.001 ran over 3 epochs`. 
- `Although a much smaller learning rate would have worked better, but it would have required more epoch runs and much more GPU hours to train. Also, a much larger learning rate would have resulted in achieving convergence too quickly resulting in suboptimal solution leading to a difficulty in reaching the global minima`. <br>

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')    # For UD Workspace
# sys.path.append('opt/cocoapi/PythonAPI')    # Personal Work Station
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import torch.optim as optim
import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size =  128          # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 500           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = optim.Adam(params, lr= 0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
loading annotations into memory...
Done (t=1.73s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.90s)
creating index...


  0%|          | 880/414113 [00:00<01:31, 4528.05it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:24<00:00, 4876.37it/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:05<00:00, 20320890.07it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

from workspace_utils import keep_awake

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in keep_awake(range(1, num_epochs+1)):
    
    for i_step in keep_awake(range(1, total_step+1)):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 3.8979, Perplexity: 49.29795
Epoch [1/3], Step [200/3236], Loss: 3.7585, Perplexity: 42.8825
Epoch [1/3], Step [300/3236], Loss: 3.1383, Perplexity: 23.0641
Epoch [1/3], Step [400/3236], Loss: 3.1200, Perplexity: 22.6454
Epoch [1/3], Step [500/3236], Loss: 2.9789, Perplexity: 19.6669
Epoch [1/3], Step [600/3236], Loss: 2.8088, Perplexity: 16.5893
Epoch [1/3], Step [700/3236], Loss: 2.8319, Perplexity: 16.9780
Epoch [1/3], Step [800/3236], Loss: 3.0867, Perplexity: 21.9041
Epoch [1/3], Step [900/3236], Loss: 2.5786, Perplexity: 13.1793
Epoch [1/3], Step [1000/3236], Loss: 2.5413, Perplexity: 12.6958
Epoch [1/3], Step [1100/3236], Loss: 2.6842, Perplexity: 14.6464
Epoch [1/3], Step [1200/3236], Loss: 2.4798, Perplexity: 11.9388
Epoch [1/3], Step [1300/3236], Loss: 2.6194, Perplexity: 13.7269
Epoch [1/3], Step [1400/3236], Loss: 2.4451, Perplexity: 11.5322
Epoch [1/3], Step [1500/3236], Loss: 2.9979, Perplexity: 20.0432
Epoch [1/3], Step [1600/3236], Lo

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [6]:
# (Optional) TODO: Validate your model.